### Bildvorverarbeitung

In [31]:
import cv2
import numpy as np
from pathlib import Path
from dataclasses import dataclass

@dataclass
class QRROIConfig:
    patch_size: int = 256
    roi_overlap: float = 0.75
    global_overlap: float = 0.75
    min_area: int = 256
    top_k: int = 50
    enable_global_search: bool = True
    global_scale_divisor: int = 4
    min_adaptive_size: int = 128
    debug_view: bool = True 

# --- Hilfsfunktionen ---

def get_square_patch(img, cx, cy, size, target_size=256):
    half = size // 2
    x0, y0 = cx - half, cy - half
    x1, y1 = x0 + size, y0 + size
    h, w = img.shape[:2]
    pad_top = max(0, -y0); pad_bottom = max(0, y1 - h)
    pad_left = max(0, -x0); pad_right = max(0, x1 - w)
    
    if any([pad_top, pad_bottom, pad_left, pad_right]):
        img_padded = cv2.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=[0,0,0])
        x0 += pad_left; x1 += pad_left; y0 += pad_top; y1 += pad_top
        patch = img_padded[y0:y1, x0:x1]
    else:
        patch = img[y0:y1, x0:x1]
        
    if patch.shape[0] != target_size or patch.shape[1] != target_size:
        patch = cv2.resize(patch, (target_size, target_size), interpolation=cv2.INTER_LANCZOS4)
    return patch

def generate_roi_patches(img, cand, cfg: QRROIConfig):
    patches, coords = [], []
    S = cfg.patch_size
    if cand["w"] <= S and cand["h"] <= S:
        crop_size = max(cand["w"], cand["h"])
        patches.append(get_square_patch(img, cand["cx"], cand["cy"], crop_size, S))
        coords.append((cand["cx"] - crop_size//2, cand["cy"] - crop_size//2, crop_size))
    else:
        stride = max(1, int(S * (1 - cfg.roi_overlap)))
        for y_s in range(cand["y"], cand["y"] + cand["h"] - S + stride, stride):
            for x_s in range(cand["x"], cand["x"] + cand["w"] - S + stride, stride):
                ax, ay = min(x_s, cand["x"] + cand["w"] - S), min(y_s, cand["y"] + cand["h"] - S)
                patches.append(get_square_patch(img, ax + S//2, ay + S//2, S, S))
                coords.append((ax, ay, S))
    return patches, coords

def generate_global_patches(img, cfg: QRROIConfig):
    h, w = img.shape[:2]
    patches, coords = [], []
    win_size = max(cfg.min_adaptive_size, min(w, h) // cfg.global_scale_divisor)
    stride = max(1, int(win_size * (1 - cfg.global_overlap)))
    for y in range(0, h - win_size + stride, stride):
        for x in range(0, w - win_size + stride, stride):
            cx, cy = min(x, w - win_size), min(y, h - win_size)
            patches.append(get_square_patch(img, cx + win_size//2, cy + win_size//2, win_size, cfg.patch_size))
            coords.append((cx, cy, win_size))
    return patches, coords

def detect_candidates(img, cfg: QRROIConfig):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
    closed = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    candidates = []
    for cnt in contours:
        if cv2.contourArea(cnt) < cfg.min_area: continue
        x, y, w, h = cv2.boundingRect(cnt)
        candidates.append({"cx": x + w // 2, "cy": y + h // 2, "x": x, "y": y, "w": w, "h": h})
    return sorted(candidates, key=lambda x: (x["w"]*x["h"]), reverse=True)[:cfg.top_k]

# --- Hauptfunktion ---

def main(input_dir, output_dir):
    cfg = QRROIConfig()
    in_path = Path(input_dir)
    out_root = Path(output_dir)
    
    # Der Hauptordner für alle Patches
    out_patches_root = out_root / "patches"
    out_patches_root.mkdir(parents=True, exist_ok=True)

    patch_metadata = []
    img_files = list(in_path.glob("*.[jJ][pP][gG]")) + list(in_path.glob("*.[pP][nN][gG]"))

    for img_file in img_files:
        img = cv2.imread(str(img_file))
        if img is None: continue
        
        # Unterordner für dieses Bild
        img_patch_dir = out_patches_root / img_file.stem
        img_patch_dir.mkdir(parents=True, exist_ok=True)
        
        h_orig, w_orig = img.shape[:2]
        vis_img = img.copy()
        
        # 1. ROI Patches
        candidates = detect_candidates(img, cfg)
        for i, cand in enumerate(candidates):
            roi_ps, roi_coords = generate_roi_patches(img, cand, cfg)
            cv2.rectangle(vis_img, (cand["x"], cand["y"]), (cand["x"]+cand["w"], cand["y"]+cand["h"]), (0, 255, 0), 4)
            
            for p_idx, (p, (px, py, ps)) in enumerate(zip(roi_ps, roi_coords)):
                name = f"ROI_{i}_p{p_idx}.jpg"
                cv2.imwrite(str(img_patch_dir / name), p)
                # relativer Pfad für die Metadaten (wichtig für Schritt 2)
                patch_metadata.append(f"{img_file.name};{img_file.stem}/{name};{px};{py};{ps};{w_orig};{h_orig}")
                cv2.rectangle(vis_img, (px, py), (px+ps, py+ps), (0, 255, 255), 2)

        # 2. Globale Patches
        if cfg.enable_global_search:
            global_ps, global_coords = generate_global_patches(img, cfg)
            for g_idx, (gp, (gx, gy, gs)) in enumerate(zip(global_ps, global_coords)):
                name = f"GLOBAL_p{g_idx}.jpg"
                cv2.imwrite(str(img_patch_dir / name), gp)
                patch_metadata.append(f"{img_file.name};{img_file.stem}/{name};{gx};{gy};{gs};{w_orig};{h_orig}")
                cv2.rectangle(vis_img, (gx, gy), (gx+gs, gy+gs), (0, 0, 255), 1)

        print(f"-> {img_file.name}: Patches in '{img_patch_dir.name}' gespeichert.")

        if cfg.debug_view:
            h, w = vis_img.shape[:2]
            scale = 800 / max(h, w)
            res_small = cv2.resize(vis_img, (int(w * scale), int(h * scale)))
            cv2.imshow("Vorschau", res_small)
            if cv2.waitKey(0) & 0xFF == ord('q'): break

    # Metadata im Hauptordner speichern
    with open(out_root / "metadata.txt", "w") as f:
        f.write("\n".join(patch_metadata))
    
    cv2.destroyAllWindows()
    print("✅ Alle Bilder verarbeitet.")

if __name__ == "__main__":
    main("test_picture", "test_patches")

-> Focus_P1_frame_570.jpg: Patches in 'Focus_P1_frame_570' gespeichert.
-> Passat_N1_frame1940.jpg: Patches in 'Passat_N1_frame1940' gespeichert.
-> Focus_N1_frame_830.jpg: Patches in 'Focus_N1_frame_830' gespeichert.
-> Focus_P1_frame_2220.jpg: Patches in 'Focus_P1_frame_2220' gespeichert.
-> Seat_N_34.jpg: Patches in 'Seat_N_34' gespeichert.
-> CKlasse_Bilderreihe 3_P_IMG_9550_dark.jpg: Patches in 'CKlasse_Bilderreihe 3_P_IMG_9550_dark' gespeichert.
-> CKlasse_Bilderreihe 2_P_IMG_9367_dark.jpg: Patches in 'CKlasse_Bilderreihe 2_P_IMG_9367_dark' gespeichert.
-> 20251129_155428.jpg: Patches in '20251129_155428' gespeichert.
-> GKLASSE_NEU_N1_frame2240.jpg: Patches in 'GKLASSE_NEU_N1_frame2240' gespeichert.
-> Focus_P2_frame_890.jpg: Patches in 'Focus_P2_frame_890' gespeichert.
-> 20251213_125103_017.jpg: Patches in '20251213_125103_017' gespeichert.
-> CKlasse_Bilderreihe 4_P_IMG_9640_rot-5.jpg: Patches in 'CKlasse_Bilderreihe 4_P_IMG_9640_rot-5' gespeichert.
-> Seat_P_46.jpg: Patches 

### Test Model

In [32]:
import os

# 1. System-Konfiguration (Unterdrückt Warnungen und behebt Berechtigungsprobleme)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['MPLCONFIGDIR'] = os.path.join(os.getcwd(), "tmp_matplotlib_cache")
if not os.path.exists(os.environ['MPLCONFIGDIR']):
    os.makedirs(os.environ['MPLCONFIGDIR'], exist_ok=True)

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
from ipywidgets import interact, IntSlider

# 2. Pfade und Parameter (Basierend auf deiner Struktur)
model_path = 'final_model.keras'
folder_path = 'test_patches/patches'
img_size = (256, 256)

# 3. Modell laden
try:
    model = load_model(model_path)
    print(f"✅ Modell '{model_path}' erfolgreich geladen.")
except Exception as e:
    print(f"❌ Fehler beim Laden des Modells: {e}")
    model = None

# 4. Bilder-Liste erstellen
if os.path.exists(folder_path):
    files = sorted([str(p.relative_to(folder_path)) for p in Path(folder_path).rglob('*') 
                    if p.suffix.lower() in ('.png', '.jpg', '.jpeg', '.tif')])
    print(f"🔍 {len(files)} Bilder in Unterordnern von '{folder_path}' gefunden.")
else:
    print(f"❌ Ordner '{folder_path}' wurde nicht gefunden!")
    files = []

# 5. Anzeige-Funktion für den Slider
def browse_patches(index):
    if not files:
        print("Keine Bilder vorhanden.")
        return

    filename = files[index]
    img_path = os.path.join(folder_path, filename)
    
    # Bild laden (Grayscale + 256x256)
    img = load_img(img_path, target_size=img_size, color_mode='grayscale')
    img_array = img_to_array(img)
    
    img_tensor = np.expand_dims(img_array, axis=0)

    # Vorhersage
    prediction = model.predict(img_tensor, verbose=0)
    
    # Bestimmung der Wahrscheinlichkeit für Klasse 1
    if prediction.shape[-1] == 1:
        prob_1 = float(prediction[0][0])
    else:
        prob_1 = float(prediction[0][1])
    
    label = 1 if prob_1 > 0.5 else 0
    color = 'green' if label == 1 else 'red'

    # Visualisierung
    plt.figure(figsize=(6, 6))
    plt.imshow(img_array.squeeze(), cmap='gray')
    
    title_str = (f"Bild {index+1}/{len(files)}: {filename}\n"
                 f"KLASSE: {label} | Wahrsch. Klasse 1: {prob_1:.4f}")
    
    plt.title(title_str, color=color, fontsize=12, fontweight='bold')
    plt.axis('off')
    plt.show()

# 6. Interaktives Element starten
if files and model:
    print("Nutze den Slider oder die Pfeiltasten deiner Tastatur zum Durchblättern:")
    interact(browse_patches, index=IntSlider(
        min=0, 
        max=len(files)-1, 
        step=1, 
        value=0, 
        description='Patch-Index:',
        layout={'width': '500px'}
    ))

✅ Modell 'final_model.keras' erfolgreich geladen.
🔍 10749 Bilder in Unterordnern von 'test_patches/patches' gefunden.
Nutze den Slider oder die Pfeiltasten deiner Tastatur zum Durchblättern:


interactive(children=(IntSlider(value=0, description='Patch-Index:', layout=Layout(width='500px'), max=10748),…

In [33]:
import cv2
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import load_model

# --- 1. GPU & System Setup (Robust) ---
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['MPLCONFIGDIR'] = str(Path.home() / ".matplotlib_cache")
Path(os.environ['MPLCONFIGDIR']).mkdir(exist_ok=True)

def setup_gpu():
    try:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                if not tf.config.experimental.get_memory_growth(gpu):
                    tf.config.experimental.set_memory_growth(gpu, True)
            print(f"✅ GPU Beschleunigung aktiv.")
    except RuntimeError:
        print("ℹ️ GPU bereits initialisiert.")
setup_gpu()

# --- 2. Konfiguration ---
class QRFinalConfig:
    model_path = 'final_model.keras'
    base_path = Path('test_patches')
    patch_folder = base_path / "patches"
    metadata_file = base_path / "metadata.txt"
    original_img_dir = Path('test_picture')
    output_dir = Path('final_results')
    
    threshold = 0.85      # Dein Schwellenwert (0.0 bis 1.0)
    batch_size = 32       # Reduziert auf 32 für stabilen VRAM

# --- 3. Die kombinierte Pipeline (Pfad A Fokus) ---
def run_combined_pipeline():
    cfg = QRFinalConfig()
    cfg.output_dir.mkdir(exist_ok=True)
    
    if not cfg.metadata_file.exists():
        print("❌ Metadaten-Datei nicht gefunden!")
        return

    model = load_model(cfg.model_path)
    
    with open(cfg.metadata_file, "r") as f:
        lines = [l.strip().split(";") for l in f.readlines()]

    images_dict = {}
    for img_name, rel_path, px, py, ps, w_orig, h_orig in lines:
        if img_name not in images_dict:
            images_dict[img_name] = {"w": int(w_orig), "h": int(h_orig), "patches": []}
        images_dict[img_name]["patches"].append({"path": rel_path, "x": int(px), "y": int(py), "s": int(ps)})

    print(f"--- Analyse startet (Pfad A: Ungefilterte Detektion) ---")

    for img_name, info in images_dict.items():
        orig_img = cv2.imread(str(cfg.original_img_dir / img_name))
        if orig_img is None: continue
        h_orig, w_orig = info["h"], info["w"]
        
        # --- BATCH LOADING (Skalierung 0-255 wie beim Training) ---
        patch_list = info["patches"]
        all_patch_imgs = []
        for p in patch_list:
            p_img = cv2.imread(str(cfg.patch_folder / p["path"]), cv2.IMREAD_GRAYSCALE)
            if p_img is None: continue
            p_img = cv2.resize(p_img, (256, 256), interpolation=cv2.INTER_LINEAR)
            all_patch_imgs.append(p_img.astype(np.float32)) # KEINE Division durch 255!

        # Vorhersage
        input_batch = np.expand_dims(np.array(all_patch_imgs), axis=-1)
        preds = model.predict(input_batch, batch_size=cfg.batch_size, verbose=0)
        
        # --- HEATMAP REKONSTRUKTION (MAX-LOGIK) ---
        heatmap_max = np.zeros((h_orig, w_orig), dtype=np.float32)
        
        max_prob = 0.0
        for i, prob_vec in enumerate(preds):
            prob = float(prob_vec[0]) if len(prob_vec) == 1 else float(prob_vec[1])
            max_prob = max(max_prob, prob)
            
            p = patch_list[i]
            x, y, s = p["x"], p["y"], p["s"]
            # Max-Pooling: Der stärkste Patch gewinnt pro Pixel
            heatmap_max[y:y+s, x:x+s] = np.maximum(heatmap_max[y:y+s, x:x+s], prob)

        # ==========================================================
        # PFAD A: DETEKTION AUF DEN ROHDATEN (KEIN FILTER!)
        # ==========================================================
        # Wir suchen die Rahmen direkt auf heatmap_max (Werte 0.0 bis 1.0)
        _, thresh = cv2.threshold(heatmap_max, cfg.threshold, 1.0, cv2.THRESH_BINARY)
        thresh_8bit = (thresh * 255).astype(np.uint8)
        
        # Konturen finden (Das sind jetzt die exakten Rahmen der Treffer-Patches)
        contours, _ = cv2.findContours(thresh_8bit, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # ==========================================================
        # PFAD B: VISUALISIERUNG (NUR FÜR DIE OPTIK)
        # ==========================================================
        heatmap_8bit = (heatmap_max * 255).astype(np.uint8)
        # Blur wird NUR für die Farbanzeige verwendet
        heatmap_vis = cv2.GaussianBlur(heatmap_8bit, (15, 15), 0)
        heatmap_color = cv2.applyColorMap(heatmap_vis, cv2.COLORMAP_JET)
        overlay = cv2.addWeighted(orig_img, 0.6, heatmap_color, 0.4, 0)
        
        # Zeichne die Rahmen aus Pfad A in das Overlay
        found_count = 0
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            # Kleiner Noise-Filter (nur Flächen größer als 15x15 px umrahmen)
            if w > 15 and h > 15:
                cv2.rectangle(overlay, (x, y), (x + w, y + h), (0, 255, 0), 4)
                found_count += 1

        # --- INFO BANNER ---
        status = "QR GEFUNDEN" if max_prob >= cfg.threshold else "KEIN TREFFER"
        cv2.rectangle(overlay, (0, 0), (overlay.shape[1], 60), (0, 0, 0), -1)
        cv2.putText(overlay, f"{status} | Max Prob: {max_prob:.1%} | Objs: {found_count}", 
                    (20, 42), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

        cv2.imwrite(str(cfg.output_dir / f"final_{img_name}"), overlay)
        print(f"✅ {img_name}: {status} ({max_prob:.2%})")

if __name__ == "__main__":
    run_combined_pipeline()

ℹ️ GPU bereits initialisiert.
--- Analyse startet (Pfad A: Ungefilterte Detektion) ---
✅ Focus_P1_frame_570.jpg: QR GEFUNDEN (100.00%)
✅ Passat_N1_frame1940.jpg: QR GEFUNDEN (99.93%)
✅ Focus_N1_frame_830.jpg: QR GEFUNDEN (92.39%)
✅ Focus_P1_frame_2220.jpg: QR GEFUNDEN (99.38%)
✅ Seat_N_34.jpg: QR GEFUNDEN (99.40%)
✅ CKlasse_Bilderreihe 3_P_IMG_9550_dark.jpg: QR GEFUNDEN (100.00%)
✅ CKlasse_Bilderreihe 2_P_IMG_9367_dark.jpg: QR GEFUNDEN (99.31%)
✅ 20251129_155428.jpg: QR GEFUNDEN (99.90%)
✅ GKLASSE_NEU_N1_frame2240.jpg: KEIN TREFFER (79.53%)
✅ Focus_P2_frame_890.jpg: QR GEFUNDEN (97.95%)
✅ 20251213_125103_017.jpg: QR GEFUNDEN (98.55%)
✅ CKlasse_Bilderreihe 4_P_IMG_9640_rot-5.jpg: QR GEFUNDEN (100.00%)
✅ Seat_P_46.jpg: QR GEFUNDEN (100.00%)
✅ GKLASSE_ALT_N1_frame2040.jpg: KEIN TREFFER (32.77%)


## Heatmap

In [34]:
import cv2
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import load_model

# --- 1. Robustes GPU-Setup ---
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def setup_gpu():
    try:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                if not tf.config.experimental.get_memory_growth(gpu):
                    tf.config.experimental.set_memory_growth(gpu, True)
            print(f"✅ GPU (RTX 4050) für Heatmap-Erstellung bereit.")
    except RuntimeError:
        print("ℹ️ GPU war bereits initialisiert. Fahre fort...")

setup_gpu()

# --- 2. Konfiguration ---
class HeatmapConfig:
    model_path = 'final_model.keras'
    base_path = Path('test_patches')
    patch_folder = base_path / "patches"
    metadata_file = base_path / "metadata.txt"
    output_folder = Path('heatmaps_output')
    
    batch_size = 32  # Passt gut in den 6GB VRAM der RTX 4050

# --- 3. Hauptfunktion ---
def generate_gpu_heatmaps():
    cfg = HeatmapConfig()
    cfg.output_folder.mkdir(exist_ok=True)

    if not cfg.metadata_file.exists():
        print(f"❌ {cfg.metadata_file} nicht gefunden!")
        return

    # Modell laden
    print("⏳ Lade Modell...")
    model = load_model(cfg.model_path)

    # Metadaten einlesen
    with open(cfg.metadata_file, "r") as f:
        data_lines = [line.strip().split(";") for line in f.readlines()]

    # Gruppierung nach Originalbild
    images_dict = {}
    for img_name, rel_patch_path, px, py, ps, w_orig, h_orig in data_lines:
        if img_name not in images_dict:
            images_dict[img_name] = {"w": int(w_orig), "h": int(h_orig), "patches": []}
        images_dict[img_name]["patches"].append({
            "rel_path": rel_patch_path, "x": int(px), "y": int(py), "s": int(ps)
        })

    print(f"--- Starte Heatmap-Generierung für {len(images_dict)} Bilder ---")

    for img_name, info in images_dict.items():
        h, w = info["h"], info["w"]
        # Initialisierung mit Nullen
        heatmap_max = np.zeros((h, w), dtype=np.float32)

        print(f"🚀 Verarbeite: {img_name}")

        # --- BATCH LOADING ---
        patch_list = info["patches"]
        all_patch_imgs = []
        
        for p in patch_list:
            # Schnelles Laden via OpenCV (Graustufen, 0-255 Scale)
            p_img = cv2.imread(str(cfg.patch_folder / p["rel_path"]), cv2.IMREAD_GRAYSCALE)
            if p_img is None: continue
            p_img = cv2.resize(p_img, (256, 256))
            all_patch_imgs.append(p_img.astype(np.float32))

        if not all_patch_imgs:
            continue

        # In Tensor-Format umwandeln (Batch, Height, Width, Channels)
        input_batch = np.expand_dims(np.array(all_patch_imgs), axis=-1)

        # --- GPU INFERENCE ---
        # Alle Patches des Bildes in einem Rutsch vorhersagen
        preds = model.predict(input_batch, batch_size=cfg.batch_size, verbose=0)

        # --- REKONSTRUKTION ---
        for i, prob_vec in enumerate(preds):
            prob = float(prob_vec[0]) if len(prob_vec) == 1 else float(prob_vec[1])
            
            p = patch_list[i]
            px, py, ps = p["x"], p["y"], p["s"]
            
            # Maximum-Logik (Pfad A): Der höchste Wert pro Pixel gewinnt
            # Das verhindert das "Verwässern" durch schlechte Patches
            heatmap_max[py:py+ps, px:px+ps] = np.maximum(heatmap_max[py:py+ps, px:px+ps], prob)

        # --- SPEICHERN ---
        # Umwandlung in 8-Bit Grayscale
        heatmap_8bit = (heatmap_max * 255).astype(np.uint8)
        
        # Optional: Falls du eine farbige Heatmap willst, aktiviere die nächste Zeile:
        # heatmap_8bit = cv2.applyColorMap(heatmap_8bit, cv2.COLORMAP_JET)

        output_path = cfg.output_folder / f"{Path(img_name).stem}_heatmap.png"
        cv2.imwrite(str(output_path), heatmap_8bit)

    print(f"✅ Alle Heatmaps wurden in '{cfg.output_folder}' erstellt.")

# --- Startbefehl ---
if __name__ == "__main__":
    generate_gpu_heatmaps()

ℹ️ GPU war bereits initialisiert. Fahre fort...
⏳ Lade Modell...
--- Starte Heatmap-Generierung für 53 Bilder ---
🚀 Verarbeite: Passat_N3_frame450.jpg
🚀 Verarbeite: Focus_P1_frame_570.jpg
🚀 Verarbeite: CKlasse_Bilderreihe 4_P_IMG_9590_rot-8.jpg
🚀 Verarbeite: Passat_N1_frame1940.jpg
🚀 Verarbeite: Focus_P1_frame_2360.jpg
🚀 Verarbeite: Focus_P1_frame_1040.jpg
🚀 Verarbeite: Focus_P1_frame_3120.jpg
🚀 Verarbeite: Passat_N3_frame2930.jpg
🚀 Verarbeite: Focus_N1_frame_830.jpg
🚀 Verarbeite: FocusP3_frame_3730.jpg
🚀 Verarbeite: Focus_P1_frame_2220.jpg
🚀 Verarbeite: 20251213_125047_068.jpg
🚀 Verarbeite: IMG_6819.jpg
🚀 Verarbeite: FocusP3_frame_1550.jpg
🚀 Verarbeite: Focus_P1_frame_500.jpg
🚀 Verarbeite: Focus_N1_frame_3250.jpg
🚀 Verarbeite: Seat_N_34.jpg
🚀 Verarbeite: FocusP4_frame_1930.jpg
🚀 Verarbeite: Focus_P1_frame_1790.jpg
🚀 Verarbeite: CKlasse_Bilderreihe 3_P_IMG_9550_dark.jpg
🚀 Verarbeite: CKlasse_Bilderreihe 2_P_IMG_9367_dark.jpg
🚀 Verarbeite: Seat_N_28.jpg
🚀 Verarbeite: GKLASSE_ALT_P1_fram

### Vergleich der erzeugten Ergebnisse

In [37]:
import cv2
import numpy as np
from pathlib import Path

# --- 1. Konfiguration der Pfade ---
path_original = Path('test_picture')
path_heatmap = Path('heatmaps_output')
path_decision = Path('final_results')

# --- 2. FIXE FENSTERGRÖSSE (Hier anpassen) ---
WINDOW_WIDTH = 1500   # Gesamtbreite des Fensters
WINDOW_HEIGHT = 500   # Gesamthöhe des Fensters

def show_fixed_triple_view():
    # Bilder suchen
    img_files = list(path_original.glob("*.[jJ][pP][gG]")) + list(path_original.glob("*.[pP][nN][gG]"))
    
    if not img_files:
        print("❌ Keine Originalbilder gefunden.")
        return

    # Einmalige Berechnung der Größe eines Einzelbildes im Triple-Stack
    single_w = WINDOW_WIDTH // 3
    single_h = WINDOW_HEIGHT

    print(f"--- Viewer gestartet (Fixe Größe: {WINDOW_WIDTH}x{WINDOW_HEIGHT}) ---")
    print("Steuerung: Beliebige Taste = Nächstes Bild | Q = Beenden")

    for img_file in img_files:
        stem = img_file.stem
        
        # 1. Bilder laden
        img_orig = cv2.imread(str(img_file))
        img_heat = cv2.imread(str(path_heatmap / f"{stem}_heatmap.png"))
        
        # Suche nach dem Ergebnisbild (verschiedene Präfixe prüfen)
        img_final = cv2.imread(str(path_decision / f"final_{img_file.name}"))
        if img_final is None:
            img_final = cv2.imread(str(path_decision / f"result_{img_file.name}"))

        # Falls ein Teil fehlt, überspringen
        if img_orig is None or img_heat is None or img_final is None:
            continue

        # 2. Resizing auf exakt die gleiche FIXE Größe
        # INTER_AREA ist am besten für das Verkleinern großer Bilder geeignet
        res_orig = cv2.resize(img_orig, (single_w, single_h), interpolation=cv2.INTER_AREA)
        res_heat = cv2.resize(img_heat, (single_w, single_h), interpolation=cv2.INTER_AREA)
        res_final = cv2.resize(img_final, (single_w, single_h), interpolation=cv2.INTER_AREA)

        # 3. Farbraum-Check (Heatmap muss 3 Kanäle haben für hstack)
        if len(res_heat.shape) == 2:
            res_heat = cv2.cvtColor(res_heat, cv2.COLOR_GRAY2BGR)

        # 4. Horizontal stapeln
        triple_view = np.hstack((res_orig, res_heat, res_final))

        # 5. Beschriftungen (statisch im Bild oben links)
        overlay_text = [
            ("ORIGINAL", 10),
            ("HEATMAP", single_w + 10),
            ("DETEKTION", (single_w * 2) + 10)
        ]

        for text, pos_x in overlay_text:
            # Kleiner schwarzer Schatten für bessere Lesbarkeit
            cv2.putText(triple_view, text, (pos_x + 2, 32), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
            # Weißer Text
            cv2.putText(triple_view, text, (pos_x, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        # 6. Anzeige
        cv2.imshow("Fixer Triple-View: QR-Analyse", triple_view)
        
        key = cv2.waitKey(0) & 0xFF
        if key == ord('q'):
            break

    cv2.destroyAllWindows()
    print("🏁 Viewer geschlossen.")

if __name__ == "__main__":
    show_fixed_triple_view()

--- Viewer gestartet (Fixe Größe: 1500x500) ---
Steuerung: Beliebige Taste = Nächstes Bild | Q = Beenden
🏁 Viewer geschlossen.
